<a href="https://colab.research.google.com/github/Weikang01/Database_reviews/blob/master/Brief_Review_On_MySQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 聚簇索引非聚簇索引
* 非聚簇：
索引被单独储存在一个B+数之中

### 不建议使用索引的情况
* 如果表的数据少，不建议使用索引（百万大概是个阈值）
* 经常增删改的表不宜使用**过多**索引（每进行一次修改，都会调整索引搜索树的结构）
* 不建议在取值范围很少的列作索引（如性别，订单状态之类）
* 二进制字段不建议当作索引

### 哪些sql语句可能导致索引的失效
* 查询条件没有使用索引
* 条件判断之中使用了不等于（<>）符号，会导致索引失效并进行全表扫描
```sql
select * from acct where acct_no <> `0001`;
```
* 使用了空值判断，会导致索引失效并进行全表扫描
```sql
select * from acct where acct_no is null;
```
* 在模糊查询中，使用了%前置会导致索引失效并进行全表扫描
```sql
select * from acct where acct_name like `%y`;
```
* 对索引进行运算，会导致索引失效并进行全表扫描
```sql
select * from acct where balance + 100 > 1000;
select * from acct where substr(acct_name,1,3) = `ABC`;
```
## **Event 事务**

> 事务的特性：ACIT
  * Atomicity原子性：事务不可分割
  * 一致性：事务提交前后从一个一致状态变为另一个一致状态
  * 隔离性：不同事务不相互影响
  * 持久性：一旦事务提交，对数据库的修改永久保存

> 事务会对数据库加锁
  * 开启事务会加锁
  * 提交事务会释放锁
  * 如果一个事务执行时间很长，是否会对数据库进行长期锁定？
    * 所以：要避免大型事务，尽量使用小事务
    * 数据库效率有叠加效应：A系统（订单系统）发送一个请求给B系统（商品系统），要求B系统在调用数据库之后返回。实际情况：如果数据库慢，整个系统都会很慢。
  * 事务会降低数据库的并行效率，但可以保证数据一致性。

#### 转账
* 转出：A 
```sql
update acct set balance = balance - 10000 where acct_no = "A";
```
* 转入：B
```sql
update acct set balance = balance + 10000 where acct_no = "B";
```

### **View 视图**
* 视图时查询数据的窗口，本身不存放数据。
* 好处：
  * 高效：复杂的连接查询，每次执行时效率比较低，新建视图，并每次从视图获取，能提高效率
```sql
create view v_acct_no_name as 
select acct_no, acct_name from acct;
```

### 什么是锁？锁的分类？
* 锁是对数据的控制权，其作用是为了解决多个工作单元对同一数据
 * 如：账号A 金额1000 -> 事务1（余额1000）：计算金额：1000 + 100 = 1100
```sql
update acct set balance = 1000 + 100 where acct_no = "A";
```
 * 事务2：取钱（余额1000）：计算金额：1000-200=800
* 如果事务1先完成，修改金额是1100，而后事务2，那么金额就变成了800，出错啦。
* 所以要加锁：必须等一个事务完成，另一个再开始

#### 锁
> 按粒度分：
  * 细粒度：行级锁：修改一条数据时，不会影响其它的数据。但也意味着这样会消耗较多的锁资源：因为要加很多锁才能锁住整张表。
  * 粗粒度：表级锁：直接锁定一整张表，并发效率低，资源消耗少。
  * infomix只支持1000万个锁，超过的话系统所有操作全部都会失败。
  * 在mysql有两种存储引擎：
    * InnoDB：
      * 两种级别都支持
    * MyISAM：
      * 不支持行级锁定，只支持表级锁定

> 按类型：
  * 读锁：共享锁：select时候加锁，加锁之后可以读取，不能写
  * 写锁：排它锁：insert/update/delete时候加锁，加锁之后可以写入，不能读、写

### 短连接、长连接
* 程序和数据库通信时需要建立连接：用完就关闭就是短连接：用完不关闭就叫长连接。短连接的优势在于占用服务器资源较少。长连接问题在于连接和断开时间消耗大。
* 连接池

## 什么是SQL执行计划
* 也就是SQL执行步骤：它可能和代码顺序并不一样，由上到下如下：
  * 执行 from 子句，找到源数据
  * where，选出满足所有条件的数据
  * group by数据分组
  * 聚合操作
  * having自居对聚合进行过滤
  * order by
  * limit对显示进行闲置

### 什么是存储引擎？
* InnoDB
  * 特性：支持事务，行级锁，外键；共享表空间
  * 适用场合：更新密集的表；有事务需求；自动灾备和回复（稳定一些）；有外键约束要求；支持自动增加列auto_increment属性。
* MyISAM
  * 特性：支持表级锁，不支持事务、外键、行锁定；访问速度快；独享表空间；数据库宕机后，MyISAM表容易损坏，灾难恢复性不佳。
  * 适用场合：查询请求较多；数据一致性要求较低；没有外键约束
* Memory
  * 特性：表结构存储于鹰派，表记录存储于内存；服务器重启后，表记录小时
  * 使用场合：数据量小；数据丢失不会造成影响；访问速度要求高。

### MySQL中有哪些跟权限相关的表？
 * user表：最重要的权限表，记录允许连接到服务器的找好信息和权限
 * db表：记录库的授权信息
 * tables_priv表：记录表的授权信息
 * columns_priv表：记录字段的授权信息

## **按要求编写SQL语句**
* 先创建一个bank数据库
```sql
create database bank default charset=utf8;
use bank;
```
* 创建一个表并且插入数据
```sql
create table acct(
  acct_no varchar(32),
  acct_name varchar(128),
  cust_no varchar(32),
  acct_type int,
  reg_date date,
  status int,
  balance decimal(16,2)
) default charset=utf8;
desc acct;
```
* 插入数据
```sql
insert into acct 
values('65135453153','Jerry','c00001',1,date(now()), 1, 5000);
select * from acct;
insert into acct values
('65135453154','Tom','c00002',1,date(now()), 1, 2000),
('65135453155','Mary','c00003',2,date(now()), 1, 7000),
('65135453156','Jack','c00004',2,date(now()), 2, 4000),
('65135453157','Lee','c00005',2,date(now()), 3, 3000);
```
* 创建客户信息表（customer,utf8），并为每个账户插入一笔信息
```sql
create table customer(
  cust_no varchar(32) primary key,
  tel_no varchar(32) not null,
  cust_name varchar(64) not null,
  address varchar(128) not null
) default charset=utf8;
select * from customer;
insert into customer values
('c00001','12312311112','Jerry',''),
('c00002','12312311112','Tom',''),
('c00003','12312311112','Mary',''),
('c00004','12312311112','Jack',''),
('c00005','12312311112','Lee',''),
```
* 查询余额大于5000的账户信息
* 账户名称以D开头
* 统计每种状态的账户笔数
* 查询最大·最小账户余额
* 查询账户余额最大的前3个账户
```sql
select * from acct where balance >= 5000 and acct_type = 2;
select * from acct where acct_name like 'D%';
select status "状态", count(*) "笔数" from acct
group by status;
select max(balance), min(balance), avg(balance), sum(balance)
from acct;
select * from acct order by balance desc limit 3;
```
* 修改acct表结构
* 给acct添加acct_no作为主键
* 给acct_name，acct_type字段添加非空约束
* 给status字段添加默认值为1
* 在reg_date字段创建普通索引
* 查询所有贷款账户（类型为2）所属客户名称、电话号码（利用子查询实现）
* 编写一个查询语句，查询结果包含：账号、户名、账户类型、余额、客户编号、客户电话、地址
```sql
alter table acct add primary key(acct_no);
desc acct;
alter table acct modify acct_name varchar(128) not null;
alter table acct modify acct_type int not null;
alter table acct modify status int default 1;
create index idx_reg_date on acct(reg_date);
select cust_name, tel_no from customer where cust_no in
(select cust_no from acct where acct_type = 2); // 找出贷款账户的客户编号
```
